## What is covered?
1. What is Langchain?
2. Sequential Langchain
3. Building Sequence of LLMChains
4. Formating Data between chains by Transform Chain
5. Building final SequentialChain
6. Dall-e 3 image genration

In [2]:
# first create and activate conda environement with the file provided as env.yaml 
import os
import json
import openai
from langchain.chains import SequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, TransformChain
from langchain.prompts import PromptTemplate
from langchain.utilities.dalle_image_generator import DallEAPIWrapper
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.getenv('OPENAI_API_KEY')

In [4]:
# defining data format for the LLM output

dataFormat = """
The JSON Object has keys 'characters', 'time-period', 'geo-location', 'scene-picture-frame', 'scene-ambience', 'original-script'\ 
Each character in characters array should have five sub keys with single values such as 'full-name', 'clothing', 'physiognomy', 'occupation', 'facical-expression', 'body-language'.\
Scene-ambience should have four sub keys 'time-of-day', 'lighting', 'environment', 'atmosphere'.\
Scene-picture-frame should two sub keys 'desciption', 'frame-picture-view' .\
time-period should be a single string value.\
geo-location should be a single string vaule focused on name of the city followed name of Kingdom/Empire/Country.\
original-script is the detailed summarization of the provided script. This field should be a single string value. You must not neglect any crucial information \
from the original script. \
"""
# prompts which we will chain together

# prompt to convert a text into a script for capturing building components of image
prompt1 = """
You are a script writer tasked with creating immersive and engaging picture of a scene. Your job is to analyze the following script scene, \
first cleaning it of any odd special characters or formatting issues. Then, expand the scene with detailed historical and cultural context, \
environmental setting, and emotional undertones. Your expansion should be rich in detail, offering inspiration for a movie director \
to finalize the scene's frames. Focus on vivid descriptions that bring the era, cultural nuances, atmosphere, and characters' historic costume, occupation, emotional states to life:\
\n\n{text}
"""

# prompt to extract image attributes such as characters, timeline, ambience from the script
prompt2 = """You are a genius in movie frame picturization, scene direction and ambience art depiction. Analyze the following movie script to identify the main characters, \
estimate the time period in which the scene is set, estimated occupation of each character in scene and their facial expressions and body language. \
For character information focus on their full name, interactions, facial expressions, relevant clothings, setting descriptions, environment, scene atmosphere and ambience,\
For scene ambience focus on the environment, time of the day, lighting effect relvant to the scene etc.\
For scene picture frame location focus on surrounding objects, infrastructure, relevant entities etc.\
Format your response as a JSON object as below:\
\n\n""" + dataFormat +"""

Analyze below script scene:
\n\n{script}
"""

# prompt to generate optimized input for dalle-3 model
prompt3 = """
You are an expert prompt writer for dall.e-3 diffusion model. Your job is to analyze below detailed information of a movie scene and characters in it to generate an image. \
You must analyze the characters, their age group, locations, best lighting, costumes to visualize the picture. You must consider the best camera angles for the picture. \
Your generated prompt must first focus on the character details and their interaction with each other then the surrondings and other scene related information. \
While creating the prompt you should consider following.\
1. Clarity and Specificity of the scene, time era with ruling Kingdom/Country etc \
2. Brevity of the intereaction between characters of the story \
3. Balance of information about surroundings and chracters \
4. Character distinct description with respect to the essense of the scene and story \
5. Avoiding overload of information \

While generating the prompt you must analyze the main characters in the scene and generate the prompt for the maximum immersion, story telling and emotions. \
Before providing the output you must checklist below requirements. \
1. The main characters should be highlighted with estimated age group distinctintly and correctly \
2. The background, ambience, historic context, time-period era with gekolocation is correctly incorported in the prompt with necessary details \
3. The prompt length is ideal for the dall.e-3 model for best results \
4. Refine the prompt again for the best output \

Your response should contain only the generated text as a string. \

analyze below information: \
\n\n{result}
"""


In [13]:
# function which sumaarizes the output of block2 (json format - dict) into a text

def process_data(input: dict) -> dict:
    """
    Process the input data and generate a final description of scene/plot..

    Args:
        input (dict): The input data containing the raw text.

    Returns:
        dict: A dictionary containing the final description.

    """
    raw_text = input["data"]
    raw_text = raw_text.replace("```json", "")
    raw_text = raw_text.replace("```", "")
    json_data = json.loads(raw_text)
    characters = json_data["characters"]
    time_period = json_data["time-period"]
    geo_location = json_data["geo-location"]
    scene_frame = json_data["scene-picture-frame"]
    ambience = json_data["scene-ambience"]
    script = json_data["original-script"]

    character_descriptions = []
    for char in characters:
        char_desc = f"{char['full-name']} wearing {char['clothing']}, a {char['occupation']} with {char['facial-expression']} and {char['body-language']}. "
        character_descriptions.append(char_desc)

    location_str = f"Locaton:\nTime period in {time_period}, set in {geo_location}."
    characters = "".join(character_descriptions)
    characters_str = f"Main characters:\n{characters}"
    ambience_str = f"Scene ambience:\nScene Setting- {ambience['time-of-day']}, Lighting- {ambience['lighting']}, Environment- {ambience['environment']}, Atmosphere- {ambience['atmosphere']}"
    frame_str = f"Scene Surroundings:\n{scene_frame['description']}, {scene_frame['frame-picture-view']}"
    script_str = f"Main Script:\n{script}"

    final_description = f"{location_str}\n\n{characters_str}\n\n{ambience_str}\n\n{frame_str}\n\n{script_str}"

    return {"result": final_description }

In [14]:
# excerpt from Harry Potter book as input to Langchain

text = """
It was a very sunny Saturday and the zoo was crowded with families. The Dursleys bought Dudley and Piers large chocolate ice creams at the entrance and then, because the smiling lady in the van had asked Harry what he wanted before they could hurry him away, they bought him a cheap lemon ice pop. It wasn't bad, either, Harry thought, licking it as they watched a gorilla scratching its head who looked remarkably like Dudley, except that it wasn't blond.

Harry had the best morning he'd had in a long time. He was careful to walk a little way apart from the Dursleys so that Dudley and Piers, who were starting to get bored with the animals by lunchtime, wouldn't fall back on their favorite hobby of hitting him. They ate in the zoo restaurant, and when Dudley had a tantrum because his knickerbocker glory didn't have enough ice cream on top, Uncle Vernon bought him another one and Harry was allowed to finish the first.

Harry felt, afterward, that he should have known it was all too good to last.

After lunch they went to the reptile house. It was cool and dark in there, with lit windows all along the walls. Behind the glass, all sorts of lizards and snakes were crawling and slithering over bits of wood and stone. Dudley and Piers wanted to see huge, poisonous cobras and thick, man-crushing pythons. Dudley quickly found the largest snake in the place. It could have wrapped its body twice around Uncle Vernon's car and crushed it into a trash can - but at the moment it didn't look in the mood. In fact, it was fast asleep.

Dudley stood with his nose pressed against the glass, staring at the glistening brown coils.

"Make it move," he whined at his father. Uncle Vernon tapped on the glass, but the snake didn't budge.

"Do it again," Dudley ordered. Uncle Vernon rapped the glass smartly with his knuckles, but the snake just snoozed on.

"This is boring," Dudley moaned. He shuffled away.

Harry moved in front of the tank and looked intently at the snake. He wouldn't have been surprised if it had died of boredom itself - no company except stupid people drumming their fingers on the glass trying to disturb it all day long. It was worse than having a cupboard as a bedroom, where the only visitor was Aunt Petunia hammering on the door to wake you up; at least he got to visit the rest of the house.

The snake suddenly opened its beady eyes. Slowly, very slowly, it raised its head until its eyes were on a level with Harry's. 
"""

In [15]:
llm_model = "gpt-4-1106-preview"
chat = ChatOpenAI(temperature=0.2, model=llm_model)


# prompt template 1: 
first_prompt = ChatPromptTemplate.from_template(prompt1)
chain_one = LLMChain(llm=chat, prompt=first_prompt, output_key="script")

# prompt template 2: 
second_prompt = ChatPromptTemplate.from_template(prompt2)
chain_two = LLMChain(llm=chat, prompt=second_prompt, output_key="data")

transform_chain = TransformChain(
    input_variables=["data"],
    output_variables=["result"],
    transform=process_data
)

third_prompt = ChatPromptTemplate.from_template(prompt3)
chain_three = LLMChain(llm=chat, prompt=third_prompt, output_key="final_prompt")

#building final chain
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, transform_chain, chain_three],
    input_variables=["text"],
    output_variables=["script","data","result", "final_prompt"],
    verbose=False)

data = overall_chain(text)
print(data['data'])

```json
{
  "characters": [
    {
      "full-name": "Harry Potter",
      "clothing": "Loose, unkempt clothing",
      "physiognomy": "Thin frame, jet-black hair, emerald eyes",
      "occupation": "Student",
      "facial-expression": "Contemplative, finding solace",
      "body-language": "Careful to maintain distance from the Dursleys, kinship with the snake"
    },
    {
      "full-name": "Vernon Dursley",
      "clothing": "Polo shirt, belted trousers",
      "physiognomy": "Broad frame",
      "occupation": "Director at a firm",
      "facial-expression": "Indifferent, demanding",
      "body-language": "Tapping on glass, authoritative"
    },
    {
      "full-name": "Petunia Dursley",
      "clothing": "Floral dress, practical bob haircut",
      "physiognomy": "Thin, sharp features",
      "occupation": "Housewife",
      "facial-expression": "Neglectful towards Harry",
      "body-language": "Non-interactive with Harry"
    },
    {
      "full-name": "Dudley Dursley",
    

In [16]:
print(data['final_prompt'])

"Capture a moment at the London Zoo in the early 1990s: Harry Potter, a boy around 11 with untidy clothes, stands contemplatively in the dimly lit reptile house, his gaze locked with a large, lethargic snake behind the glass. Nearby, Vernon Dursley, a stern man in his 40s wearing a polo shirt and belted trousers, taps authoritatively on the glass, while his son Dudley, a spoiled child around 11 with ice cream stains on his clothes, looks on bored and whiny. Petunia Dursley, indifferent, stands in the background in a floral dress and practical bob haircut. The Victorian architecture of the reptile house looms around them, with soft eerie light filtering through, contrasting the bright, carefree atmosphere outside. The scene is a blend of Harry's quiet kinship with the snake and the oppressive indifference of the Dursleys."


In [17]:
disable_prompt_rewrite = "Do not revise provided prompt delimited by double quotes. DO NOT add any details, just use it AS-IS.:"
prompt = f"{disable_prompt_rewrite}\n{data['final_prompt']}"
print(prompt)

response = openai.Image.create(
  model="dall-e-3",
  prompt=prompt,
  size="1792x1024",
  quality="hd",
  style="vivid",
  n=1,
)


Do not revise provided prompt delimited by double quotes. DO NOT add any details, just use it AS-IS.:
"Capture a moment at the London Zoo in the early 1990s: Harry Potter, a boy around 11 with untidy clothes, stands contemplatively in the dimly lit reptile house, his gaze locked with a large, lethargic snake behind the glass. Nearby, Vernon Dursley, a stern man in his 40s wearing a polo shirt and belted trousers, taps authoritatively on the glass, while his son Dudley, a spoiled child around 11 with ice cream stains on his clothes, looks on bored and whiny. Petunia Dursley, indifferent, stands in the background in a floral dress and practical bob haircut. The Victorian architecture of the reptile house looms around them, with soft eerie light filtering through, contrasting the bright, carefree atmosphere outside. The scene is a blend of Harry's quiet kinship with the snake and the oppressive indifference of the Dursleys."


In [18]:
from IPython.display import Image, display
display(Image(url=response["data"][0]['url']))

### Conclusion
In this article, we demonstrated a basic implementation of the LangChain framework, using it to convert text into DALL-E 3 prompts for generating realistic and historically accurate artistic images. Our exploration into this field will continue with more advanced RAG pipeline applications in our next piece.

**🌟 Connect on LinkedIn!** 

If you've found this content _useful_ and would like to explore more about **data science**, **machine learning**, and related fields, I'd be delighted to see you on my LinkedIn network. I share insights, resources, and the latest trends that could be beneficial for your learning journey.

➤ [**_Follow on LinkedIn_**](https://www.linkedin.com/in/aambekar234/)

_Your support and interaction are always appreciated._

**Best Regards,**
**Abhijeet Ambekar**